### Training the benchmarking models 

- To train the benchmarking models, simply run the following command on a terminal from within this folder: <br> <code>python benchmarking.py 'DeepSEA' </code> 
- If you would like to save the model, you can change the save flag to 1 in `line 16` of `benchmarking.py` 
- The code for benchmarking.py (which was used for training the benchmarking models) can also be inspected in the last section of this notebook. 
- Benchmarking.py trains a model from scratch, loads the random test data (corresponding to Fig. 1a), generates predictions and saves the model weights and the predictions for this data. 
- The resutls can also be generated from saved models using Notebook 1 from this folder.

### Computing the ECC using the benchmarking models

#### Load the chosen benchmarking model ( DeepSEA / DeepAtt / DanQ )
The saved model is available in this directory under the name `[model_name].h5`


##### Note about the final layer :

We emphasize that the final layer does have a <b>single output unit and a linear activation</b> as one would expect. This is can be verified by looking at the following line in the `benchmarking.py` file in this folder and also in the cell below : <br>
<code>output_layer = Dense(1, kernel_regularizer = l1_l2(l1=l1_weight, l2=l2_weight),
                     activation='linear', kernel_initializer='he_normal', use_bias=True )(x) 
</code>


#####  Note about the DeepAtt() model : 

- The reader will notice that there is a fitness_function_model() call instead of DeepAtt() for the model_name = 'DeepAtt'. The reason is that there are some non JSON-serializable layers in the DeepAtt() function invocation (from their original authors) which makes it difficult to save using model.save(). The model can be trained (using notebook 2 in this folder) but the jupyter notebook needs to be kept running for the complete analysis (including computing predictions and computing ECC using this model) since model.save_weights() results in errors when loading and model.save() will not work. 
- The ECC for DeepAtt was also similarly calculated using this single running notebook from end to end using the DeepAtt() call to stay faithful to the original implementation. We also save a model trained using our own implementation of the DeepAtt() model with JSON-seriable layers for future predictions.
- So, to save a version of the 'DeepAtt' model, we use our own layers to address the issues we had in saving with the implementation in DeepAtt() from the original authors. 
- It can be verified that both lead to equivalent performance by changing the 'define_DeepAtt_here' flag in Notebook 2 and the bechmarking.py file.
- The DeepAtt() model was trained from scratch here again (in this same VM and environment) using both approaches (our adapted approach and the original authors approach) to verify the equivalent peformance and it worked as expected consistently.


In [1]:
model_name = 'DeepSEA_model'#'DanQ_model' , 'DeepSEA_model' , 'DeepAtt_model'


import BioinforDeepATT
from BioinforDeepATT.model.model import *
import sys
sys.path.insert(0, './')
#%load_ext autoreload
#%autoreload 2
from rr_aux import *


##Clear Memory 
#tf.reset_default_graph()
tf.keras.backend.clear_session()
gc.collect()
##



if model_name in ['DanQ_model' , 'DeepSEA_model']:
    model = tf.keras.models.load_model(model_name+'.h5')
else :
    

    def fitness_function_model(model_params) :

        batch_size= model_params['batch_size']
        l1_weight= model_params['l1_weight']
        l2_weight= model_params['l2_weight']
        lr= model_params['lr']
        device_type = model_params['device_type']
        input_shape = model_params['input_shape']
        loss = model_params['loss']
        model_name = model_params['model_name']

        if(model_params['device_type']=='tpu'):
            input_layer = Input(batch_shape=(batch_size,input_shape[1],input_shape[2]))  #trX.shape[1:] #batch_shape=(batch_size,110,4)

        else :
            input_layer = Input(shape = input_shape[1:] , batch_size = 1024)  #trX.shape[1:] #

        if model_name=='DeepAtt':
            if 1 : 
                x = Conv1D(256, 30, padding='valid' ,\
                       kernel_regularizer  = l1_l2(l1=l1_weight, l2=l2_weight), kernel_initializer='he_normal' ,
                      data_format = 'channels_last' , activation='relu')(input_layer) 
                x = tf.keras.layers.MaxPool1D( pool_size=3, strides=3, padding='valid')(x)
                x= tf.keras.layers.Dropout(0.2)(x)
                x = Bidirectional(LSTM(16, return_sequences=True,kernel_initializer='he_normal'))(x)
                x = MultiHeadAttention( head_num=16)(x) 
                x = tf.keras.layers.Dropout(0.2)(x)
                x = Flatten()(x)

                x = keras.layers.Dense(
                        units=16,
                        activation='relu')(x)
                x = keras.layers.Dense(
                    units=16,
                    activation='relu')(x)
            else : 
                x = DeepAtt()
                x = x.call(input_layer)

        if model_name=='DanQ':
            x = DanQ()
            x = x.call(input_layer)

        if model_name=='DeepSEA':
            x = DeepSEA()
            x = x.call(input_layer)

        if(len(x.get_shape())>2):
            x = Flatten()(x) 

        output_layer = Dense(1, kernel_regularizer = l1_l2(l1=l1_weight, l2=l2_weight),
                        activation='linear', kernel_initializer='he_normal', use_bias=True )(x) 


        model = Model(input_layer, output_layer)
        opt = tf.compat.v1.train.AdamOptimizer(lr) #tf.keras.optimizers.Adam(lr=lr)#


        model.compile(optimizer=opt, loss=loss,metrics=['mean_squared_error', 'cosine_similarity']) 

        return model

    model_params = {
            'n_val_epoch' : 1000,
            'epochs' : 3, #3 used previously
            'batch_size': int(1024*1), # int(1024*3) , #64*55 fits , #best batch size is 1024
            'l1_weight': 0,#1e-6#1e-7#0.01 # l1 should always be zero
            'l2_weight': 0,#1e-7#0.01
            'lr':0.001,
            'device_type' : 'gpu', #'tpu'/'gpu'/'cpu'
            'input_shape' : (30722048, 110, 4),
            'loss' : 'mean_squared_error', 
            'model_name' : 'DeepAtt'}

    model=fitness_function_model(model_params)
    model.load_weights('DeepAtt')
scaler= sklearn.externals.joblib.load(os.path.join('..','..','..','data','Glu','scaler.save'))
batch_size = 1024

/home/edv/anaconda3/envs/evolution/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/edv/anaconda3/envs/evolution/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/edv/anaconda3/envs/evolution/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/edv/anaconda3/envs/evolution/lib/python3

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


/home/edv/anaconda3/envs/evolution/lib/python3.7/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator StandardScaler from version 0.20.3 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


#### Define the helper functions

In [2]:
args  = {'sequence_length' : 80}
np.random.seed(0)

#########################################################################################################
#########################################################################################################
### Thanks, Stack Overflow ! 
def mean_(val, freq):
    return np.average(val, weights = freq)

def median_(val, freq):
    ord_ = np.argsort(val)
    cdf = np.cumsum(freq[ord_])
    return val[ord_][np.searchsorted(cdf[-1] // 2, cdf)]

def mode_(val, freq): #in the strictest sense, assuming unique mode
    return val[np.argmax(freq)]

def var_(val, freq):
    if(len(val)==1):
        return 0
    avg = mean_(val, freq)
    dev = freq * (val - avg) ** 2
    return dev.sum() / (freq.sum() - 1)

def std_(val, freq):
    return np.sqrt(var_(val, freq))


#########################################################################################################
def freq_df(pro_ortho_sequences):
    ### Returns : Base Frequency distribution for this set of sequences
    #Verified against : http://hplgit.github.io/bioinf-py/doc/pub/html/main_bioinf.html
    pro_freq_df = pd.DataFrame(index=range(len(pro_ortho_sequences[0])) , columns=['A','C','G','T'] ,
                              data = np.zeros( (len(pro_ortho_sequences[0]),4)))
    for sequence in pro_ortho_sequences : 
        for index,base in enumerate(sequence) : 
            pro_freq_df.loc[index,base]+=1
            
    pro_freq_df = pro_freq_df.div(pro_freq_df.sum(axis=1), axis=0)

    return pro_freq_df

#########################################################################################################
#########################################################################################################
### Generate all possible single mutations in population : population_next 


def population_mutator( population_current , args) :
    population_current = population_remove_flank(population_current)
    population_next = []  
    for i in range(len(population_current)) :         
        for j in range(args['sequence_length']) : 
        #First create three copies of the same individual, one for each possible mutation at the basepair.
            population_next.append(list(population_current[i]))
            population_next.append(list(population_current[i]))
            population_next.append(list(population_current[i]))
            
            if (population_current[i][j] == 'A') :
                population_next[3*(args['sequence_length']*i + j) ][j] = 'C'
                population_next[3*(args['sequence_length']*i + j) + 1][j] = 'G'
                population_next[3*(args['sequence_length']*i + j) + 2][j] = 'T'
                
            elif (population_current[i][j] == 'C') :
                population_next[3*(args['sequence_length']*i + j)][j] = 'A'
                population_next[3*(args['sequence_length']*i + j) + 1][j] = 'G'
                population_next[3*(args['sequence_length']*i + j) + 2][j] = 'T'
            
            elif (population_current[i][j] == 'G') :
                population_next[3*(args['sequence_length']*i + j)][j] = 'C'
                population_next[3*(args['sequence_length']*i + j) + 1][j] = 'A'
                population_next[3*(args['sequence_length']*i + j) + 2][j] = 'T'
                
            elif (population_current[i][j] == 'T') :
                population_next[3*(args['sequence_length']*i + j)][j] = 'C'
                population_next[3*(args['sequence_length']*i + j) + 1][j] = 'G'
                population_next[3*(args['sequence_length']*i + j) + 2][j] = 'A'
             
        
    population_next= population_add_flank(population_next)        
    return list(population_next)





#########################################################################################################
#########################################################################################################
#########################################################################################################




#######################################################################################
#######################################################################################
#######################################################################################

def closest_point(node, nodes):
    nodes = np.asarray(nodes)
    deltas = nodes - node
    dist_2 = np.einsum('ij,ij->i', deltas, deltas)
    return np.argmin(dist_2)

#######################################################################################
#######################################################################################
#######################################################################################

def get_snpdev_dist(population) : 
    population_fitness = np.array(evaluate_model(list(population),model,scaler,batch_size))
    args  = {'sequence_length' : 80 , 'nucleotide_frequency' :[0.25,0.25,0.25,0.25] , 'randomizer' : np.random } 
    population_1bp_all_sequences = population_mutator(list(population) , args)
    population_1bp_all_fitness = np.array(evaluate_model(list(population_1bp_all_sequences),model,scaler,batch_size))


    snpdev_dist = []
    for i in (range(len(population))) :   
        original_fitness = population_fitness[i]
        sequence = population[i]

        exp_dist = population_1bp_all_fitness[3*args['sequence_length']*i:3*args['sequence_length']*(i+1)]
        snpdev_dist = snpdev_dist + [np.sort((exp_dist-original_fitness))]

    sequences = population
    return snpdev_dist


#######################################################################################
#######################################################################################
#######################################################################################


def analyze_seq(population ) : 
    ### Quick function to generate an analyzed dataframe for any sequence. Only works in this NB.

    data_at = AAnet_model.data2at(get_snpdev_dist(population))
    Y_mds_data = data_at @ Y_mds_ats
    
    data_at_df = pd.DataFrame(data_at , columns = ['AT1','AT2','AT3']).join(
    pd.DataFrame(Y_mds_data , columns = ['MDS1','MDS2'])).join(
    pd.DataFrame(evaluate_model(population,model,scaler,batch_size),
                 columns = [model_conditions+'_exp']))
    return data_at_df, data_at , Y_mds_data

#######################################################################################
#######################################################################################
#######################################################################################


def snpdev_str_to_list(snpdev_str) : 
    return [float(i) for i in snpdev_str.replace("\n" , "").replace("[","").replace("]","").split()]

def name2sys(gene_name):
    return yeast_genome_annotations[yeast_genome_annotations.gene_name==gene_name].systematic_name.values[0]
#########################################################################################################
%matplotlib inline

#### Read the ortholog sequences

In [3]:


ortho_sequences_df = pd.read_csv('20190719_Native80.2_orthologous_promoters_unique.txt.gz', sep='\t')
ortho_sequences_df = ortho_sequences_df[~ortho_sequences_df.ortho_seq.str.contains('N')]
nonunique_ortho_sequences_df = pd.read_csv('20190719_Native80.2_orthologous_promoters.txt.gz', sep='\t')
nonunique_ortho_sequences_df = nonunique_ortho_sequences_df[~nonunique_ortho_sequences_df.ortho_seq.str.contains('N')]
### Compute Expression
ortho_sequences_df['EL'] = evaluate_model(list(ortho_sequences_df.ortho_seq), 
                                                  model,scaler,batch_size)

ortho_sequences = list(ortho_sequences_df['ortho_seq'].values)
if len(ortho_sequences[0])==80 :
    ortho_sequences = population_add_flank(list(ortho_sequences))

### List of Unique Promoters
proName_list = np.unique(ortho_sequences_df.proName.values)

#! cp ../../figure_orthologs/pro_consensus_dict.npy .
pro_consensus_dict = np.load('pro_consensus_dict.npy', allow_pickle=1).item()
proName_list = list(pro_consensus_dict.keys())


62464/62464 [==============================] - 46s 730us/sample


#### Compute the number of sequences at each hamming distance from in the 1011 orthologs data

In [4]:
def find_k(i):
    seq= nonunique_ortho_sequences_df.ortho_seq[i]
    reference = pro_consensus_dict[nonunique_ortho_sequences_df.proName[i]] # pro_df.loc[nonunique_ortho_sequences_df.proName[i]].reference_sequence#
    return int(hamming_distance(seq , reference))

num_cores = multiprocessing.cpu_count()-5
pool = mp.Pool(num_cores)
nonunique_ortho_sequences_df['k'] = pool.map(  find_k,  (nonunique_ortho_sequences_df.index))
pool.close()
pool.join()



def find_k_unique(i):
    seq= ortho_sequences_df.ortho_seq[i]
    reference = pro_consensus_dict[ortho_sequences_df.proName[i]] #pro_df.loc[ortho_sequences_df.proName[i]].reference_sequence#
    return int(hamming_distance(seq , reference))

num_cores = multiprocessing.cpu_count()-5
pool = mp.Pool(num_cores)
ortho_sequences_df['k'] = pool.map(  find_k_unique,  (ortho_sequences_df.index))
pool.close()
pool.join()

ortho_sequences_df['EL'] = evaluate_model(list(ortho_sequences_df.ortho_seq), 
                                                  model,scaler,batch_size)
nonunique_ortho_sequences_df['EL'] = nonunique_ortho_sequences_df.ortho_seq.map(
    dict(zip(ortho_sequences_df.ortho_seq,ortho_sequences_df.EL)))
#ortho_sequences_df.to_csv(model_name+'ortho_sequences_df_analyzed.tsv', sep='\t')
#nonunique_ortho_sequences_df.to_csv(model_name+'nonunique_ortho_sequences_df_analyzed.tsv', sep='\t')

62464/62464 [==============================] - 44s 712us/sample


#### Compute the var

In [5]:
pro_df = pd.DataFrame(index=proName_list , columns = {'var_EL_actual','var_EL_sim'})
for i in tqdm(proName_list):
    ortho_subset= ortho_sequences_df[ortho_sequences_df.proName==i]
    pro_df.loc[i,'var_EL_actual'] = var_(ortho_subset.EL , ortho_subset.num_isolates)
    #pro_df.loc[i,'var_EL_actual_unweighted'] = np.var(ortho_subset.EL)

ref_df=pd.read_csv(os.path.join('..','..','..','data','native_sequences_only','allNativeChunks.all') , 
            header=None , sep='\t',index_col=0 , names=['reference_sequence'])#.loc[pro_df.index].values


pro_df = pro_df.join(ref_df)
pro_stats = pd.read_csv('20190719_Native80.2_promoter_divergence.txt',
                   sep='\t' , index_col=0)
pro_df = pro_df.join(pro_stats )

#if 0 : 
#pro_df = pd.read_csv('./pro_df_analyzed.tsv',sep='\t', index_col=0)

proName_list  = pro_df.index.values

100%|██████████| 5569/5569 [00:26<00:00, 210.02it/s]


#### Compute the ECC 
`log_ratio` field in the table corresponds to the ECC.<br>
For more detail on ECC calculation, please see: https://github.com/1edv/evolution/blob/master/manuscript_code/ecc_mr_fr/ecc_mr_fr.ipynb

In [6]:

#! cp ../../figure_orthologs/sim_unique_df_full .
sim_unique_df_full = pd.read_csv('./sim_unique_df_full',sep='\t', index_col=0)
#! cp ../../figure_orthologs/sim_unique_df_full_v2 .
sim_unique_df_full_v2 = pd.read_csv('./sim_unique_df_full_v2',sep='\t', index_col=0)

sim_unique_df_full['EL'] = evaluate_model(list(sim_unique_df_full.sim_seq), model,scaler,batch_size)
sim_unique_df_full_v2['EL'] = evaluate_model(list(sim_unique_df_full_v2.sim_seq), model,scaler,batch_size)


for pro in tqdm(proName_list):
    sim_subset= sim_unique_df_full[sim_unique_df_full.proName==pro]
    pro_df.loc[pro,'std_EL_sim'] = std_(sim_subset.EL , sim_subset.num_isolates) #STD not var, ignore name
    pro_df.loc[pro,'var_EL_sim'] = var_(sim_subset.EL , sim_subset.num_isolates) #STD not var, ignore name

    sim_subset_v2= sim_unique_df_full_v2[sim_unique_df_full_v2.proName==pro]
    pro_df.loc[pro,'std_EL_sim_v2'] = std_(sim_subset_v2.EL , sim_subset_v2.num_isolates)#STD not var, ignore name
    pro_df.loc[pro,'var_EL_sim_v2'] = var_(sim_subset_v2.EL , sim_subset_v2.num_isolates)#STD not var, ignore name

    ortho_subset= ortho_sequences_df[ortho_sequences_df.proName==pro]
    pro_df.loc[pro,'std_EL_actual'] = std_(ortho_subset.EL , ortho_subset.num_isolates)#STD not var, ignore name
    pro_df.loc[pro,'var_EL_actual'] = var_(ortho_subset.EL , ortho_subset.num_isolates)#STD not var, ignore name


pro_df['uncorrected_ratio'] =  (pro_df.std_EL_sim+(np.finfo(float).eps)) / (pro_df.std_EL_actual +(np.finfo(float).eps))
pro_df['uncorrected_log_ratio'] = pro_df.uncorrected_ratio.astype('float').apply(np.log2)

pro_df['ratio_v2'] =  (pro_df.std_EL_sim_v2+(np.finfo(float).eps)) / (pro_df.std_EL_actual +(np.finfo(float).eps))
pro_df['log_ratio_v2'] = pro_df.ratio_v2.astype('float').apply(np.log2)

pro_consensus_dict_df = pd.DataFrame.from_dict(pro_consensus_dict , orient='index')
pro_consensus_dict_df.columns = ['consensus_sequence']
pro_df = pro_df.join(pro_consensus_dict_df)

pro_ref_pred_EL_df = pd.read_csv(os.path.join('..','..','..','data','native_sequences_only','pro_ref_pred_SC_URA_GLU_EL.tsv'), sep='\t' ,
               index_col = 0 )
pro_df = pro_df.join(pro_ref_pred_EL_df.loc[pro_df.index , ['pred_ref_SC_Ura_EL' , 'pred_ref_Glu_EL']])

pro_df.loc[pro_df.var_EL_actual==0,'uncorrected_log_ratio']=np.nan
pro_df.loc[pro_df.var_EL_actual==0,'uncorrected_ratio']=np.nan

pro_df.loc[pro_df.var_EL_actual==0,'log_ratio']=np.nan
pro_df.loc[pro_df.var_EL_actual==0,'ratio']=np.nan

pro_df.loc[pro_df.var_EL_actual==0,'log_ratio_v2']=np.nan
pro_df.loc[pro_df.var_EL_actual==0,'ratio_v2']=np.nan


pro_df['ratio_sim_sim'] =  (pro_df.std_EL_sim+(np.finfo(float).eps)) / (pro_df.std_EL_sim_v2 +(np.finfo(float).eps))
pro_df['log_ratio_sim_sim'] = pro_df.ratio_sim_sim.astype('float').apply(np.log2)
sim_sim_median = np.median(pro_df['log_ratio_sim_sim'])

pro_df['log_ratio'] = pro_df['uncorrected_log_ratio'] - sim_sim_median


if 0 : ### change to 1 if you would like to save the ECC results using the alternative model

    pro_df.to_csv(model_name+'_pro_df_analyzed.tsv',sep='\t')


62464/62464 [==============================] - 42s 666us/sample


100%|██████████| 5569/5569 [04:34<00:00, 20.28it/s]


In [7]:
### This code block computes mutation tolerance
if 0 : 
    cons_df = pro_df[pro_df.log_ratio>0]
    epsilon = 2*std_(cons_df.var_EL_actual.values.astype(np.float64) ,cons_df.num_valid_hamming.values )
    print('Epsilon : ' + str(epsilon))
    reference_snpdev_dist = get_snpdev_dist(population_add_flank(list(pro_df.reference_sequence)))
    consensus_snpdev_dist = get_snpdev_dist(population_add_flank(list(pro_df.consensus_sequence)))

    mutation_tolerance = ((np.abs(consensus_snpdev_dist) -epsilon)<0).sum(axis=1) / len(consensus_snpdev_dist[0])
    pro_df['mutation_tolerance']=mutation_tolerance

    if 0 : ### change to 1 if you would like to save the ECC results using the alternative model
        pro_df.to_csv(model_name+'_pro_df_analyzed.tsv',sep='\t')


### Code for training the benchmarking models 

#### benchmarking.py was used to run the following code blocks , the cells below are just for convenient reading
#### Benchmarking.py trains a model, loads the random test data (corresponding to Fig. 1a), generates predictions and saves the model weights and the predictions for this data.

In [ ]:
import BioinforDeepATT
from BioinforDeepATT.model.model import *
import sys
sys.path.insert(0, './')
%load_ext autoreload
%autoreload 2
from rr_aux import *


##Clear Memory 
tf.reset_default_graph()
tf.keras.backend.clear_session()
gc.collect()
##

define_DeepAtt_here = 1 
save = 0

In [ ]:


NUM_GPU = len(get_available_gpus())
if(NUM_GPU>0):
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU

print(tf.__version__)
print(keras.__version__)

model_conditions = 'Glu'


#sys.argv[1] appended to beginning of path
dir_path=os.path.join('..','..','..','data',model_conditions)


###





## Load the data matrix
with h5py.File(join(dir_path,'_trX.h5'), 'r') as hf:
    _trX = hf['_trX'][:]

with h5py.File(join(dir_path,'_trY.h5'), 'r') as hf:
    _trY = hf['_trY'][:]

with h5py.File(join(dir_path,'_vaX.h5'), 'r') as hf:
    _vaX = hf['_vaX'][:]

with h5py.File(join(dir_path,'_vaY.h5'), 'r') as hf:
    _vaY = hf['_vaY'][:]

with h5py.File(join(dir_path,'_teX.h5'), 'r') as hf:
    _teX = hf['_teX'][:]

with h5py.File(join(dir_path,'_teY.h5'), 'r') as hf:
    _teY = hf['_teY'][:]



_trX.shape , _trY.shape , _vaX.shape , _vaY.shape  , _teX.shape , _teY.shape



trX = _trX[:int(_trX.shape[0]/1024)*1024] #np.concatenate((_trX, _trX_rc), axis = 1) #np.squeeze((_trX))#
vaX = _vaX[:int(_vaX.shape[0]/1024)*1024] # np.concatenate((_vaX, _vaX_rc), axis = 1) #np.squeeze((_vaX))#
teX = _teX[:int(_teX.shape[0]/1024)*1024] # np.concatenate((_teX, _teX_rc), axis = 1)#np.squeeze((_teX))#



## Load the scaler function (scaler was trained on the synthesized data
scaler = sklearn.externals.joblib.load(join(dir_path,'scaler.save')) 




vaY = (scaler.transform(_vaY.reshape(1, -1))).reshape(_vaY.shape)[:int(_vaY.shape[0]/1024)*1024] #_vaY#
trY = (scaler.transform(_trY.reshape(1, -1))).reshape(_trY.shape)[:int(_trY.shape[0]/1024)*1024] #_trY#
teY = (scaler.transform(_teY.reshape(1, -1))).reshape(_teY.shape)[:int(_teY.shape[0]/1024)*1024] #_teY#


### If using generator, have a smaller val set for faster evaluation
if 0: 
    s_trX = np.vstack((trX , vaX))
    s_trY = np.vstack((trY , vaY))
    trX = s_trX[1000:,:]
    trY = s_trY[1000:,:]
    vaX = s_trX[0:1000,:]
    vaY = s_trY[0:1000,:]

print(trX.shape , trY.shape , vaX.shape , vaY.shape  , _teX.shape , _teY.shape)

input_shape = trX.shape


In [ ]:
import BioinforDeepATT
from BioinforDeepATT.model.model import *
if define_DeepAtt_here == 1 :
    del Bidirectional, LSTM
from rr_aux import *  ### Overrides DeepAtt() imports

def fitness_function_model(model_params) :

    batch_size= model_params['batch_size']
    l1_weight= model_params['l1_weight']
    l2_weight= model_params['l2_weight']
    lr= model_params['lr']
    device_type = model_params['device_type']
    input_shape = model_params['input_shape']
    loss = model_params['loss']
    model_name = model_params['model_name']
    
    if(model_params['device_type']=='tpu'):
        input_layer = Input(batch_shape=(batch_size,input_shape[1],input_shape[2]))  #trX.shape[1:] #batch_shape=(batch_size,110,4)

    else :
        input_layer = Input(shape = input_shape[1:] , batch_size = 1024)  #trX.shape[1:] #

    if model_name=='DeepAtt':
        if define_DeepAtt_here : 
            x = Conv1D(256, 30, padding='valid' ,\
                   kernel_regularizer  = l1_l2(l1=l1_weight, l2=l2_weight), kernel_initializer='he_normal' ,
                  data_format = 'channels_last' , activation='relu')(input_layer) 
            x = tf.keras.layers.MaxPool1D( pool_size=3, strides=3, padding='valid')(x)
            x= tf.keras.layers.Dropout(0.2)(x)
            x = Bidirectional(LSTM(16, return_sequences=True,kernel_initializer='he_normal'))(x)
            x = MultiHeadAttention( head_num=16)(x) 
            x = tf.keras.layers.Dropout(0.2)(x)
            x = Flatten()(x)

            x = keras.layers.Dense(
                    units=16,
                    activation='relu')(x)
            x = keras.layers.Dense(
                units=16,
                activation='relu')(x)
        else : 
            x = DeepAtt()
            x = x.call(input_layer)

    if model_name=='DanQ':
        x = DanQ()
        x = x.call(input_layer)

    if model_name=='DeepSEA':
        x = DeepSEA()
        x = x.call(input_layer)

    if(len(x.get_shape())>2):
        x = Flatten()(x) 
        
    output_layer = Dense(1, kernel_regularizer = l1_l2(l1=l1_weight, l2=l2_weight),
                    activation='linear', kernel_initializer='he_normal', use_bias=True )(x) 


    model = Model(input_layer, output_layer)
    opt = tf.compat.v1.train.AdamOptimizer(lr) #tf.keras.optimizers.Adam(lr=lr)#
    

    model.compile(optimizer=opt, loss=loss,metrics=['mean_squared_error', 'cosine_similarity']) 
    
    return model


In [ ]:
model_name_list = sys.argv#['DeepSEA' , 'DanQ' , 'DeepAtt'  ]
model_name = 'DeepSEA'#model_name_list[0]
model_params = {
    'n_val_epoch' : 1000,
    'epochs' : 3,
    'batch_size': int(1024*1), # int(1024*3) , #64*55 fits , #best batch size is 1024
    'l1_weight': 0,#1e-6#1e-7#0.01 # l1 should always be zero
    'l2_weight': 0,#1e-7#0.01
    'lr':0.001,
    'device_type' : 'gpu', #'tpu'/'gpu'/'cpu'
    'input_shape' : input_shape,#input_shape,
    'loss' : 'mean_squared_error', 
    'model_name' : model_name}
epochs = model_params['epochs']  
batch_size =  model_params['batch_size']
n_val_epoch =  model_params['n_val_epoch']
epochs = model_params['epochs']



if 0 : 
    ### Save model params as csv
    w = csv.writer(open(os.path.join(model_name+'_model_params.csv'), "w"))
    for key, val in model_params.items():
        w.writerow([key, val])

    ### Save model params as pickle
    f = open(os.path.join(model_name+'_model_params.pkl'),"wb")
    pickle.dump(model_params,f)
    f.close()


model=fitness_function_model(model_params)

print(model.summary())

model.fit(trX, trY, validation_data = (vaX[:1024], vaY[:1024]), batch_size=batch_size  , epochs=epochs  )
#model.fit_generator(training_generator, validation_data = (teX[:100], teY[:100]),
#epochs=epochs , steps_per_epoch = int(trX.shape[0]/(batch_size*n_val_epoch)) )



def read_hq_testdata(filename) :

    with open(filename) as f:
        reader = csv.reader(f, delimiter="\t")
        d = list(reader)

    sequences = [di[0] for di in d]

    for i in tqdm(range(0,len(sequences))) : 
        if (len(sequences[i]) > 110) :
            sequences[i] = sequences[i][-110:]
        if (len(sequences[i]) < 110) : 
            while (len(sequences[i]) < 110) :
                sequences[i] = 'N'+sequences[i]



    A_onehot = np.array([1,0,0,0] ,  dtype=np.bool)
    C_onehot = np.array([0,1,0,0] ,  dtype=np.bool)
    G_onehot = np.array([0,0,1,0] ,  dtype=np.bool)
    T_onehot = np.array([0,0,0,1] ,  dtype=np.bool)
    N_onehot = np.array([0,0,0,0] ,  dtype=np.bool)

    mapper = {'A':A_onehot,'C':C_onehot,'G':G_onehot,'T':T_onehot,'N':N_onehot}
    worddim = len(mapper['A'])
    seqdata = np.asarray(sequences)
    seqdata_transformed = seq2feature(seqdata)
    print(seqdata_transformed.shape)


    expressions = [di[1] for di in d]
    expdata = np.asarray(expressions)
    expdata = expdata.astype('float')  

    return np.squeeze(seqdata_transformed),expdata

X,Y = read_hq_testdata(os.path.join('..','..','..','data','Glu','HQ_testdata.txt'))
Y = [float(x) for x in Y]
Y_pred= evaluate_model(X, model, scaler)

pcc = scipy.stats.pearsonr(Y,Y_pred )[0]
print(pcc)

if save : 
    df = pd.DataFrame({'Y' : Y , 'Y_pred' : Y_pred , 'pcc' : pcc})
    df.to_csv(model_name+"_results.csv")
    model.save(model_name+"_model.h5")
    model.save_weights(model_name)
